## Tutorial: Create an Isolated VLAN and Servers

This notebook demonstrates how to create isolated VLANs and attach servers.

#### Modules 

- [Reserve a Mulitple Resources](../modules-python/reservations/reserve_multiple_resources.ipynb)
- [Get Lease by Name](../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../modules-python/reservations/get_reservations_from_lease.ipynb)
- [Create Server](../modules-python/servers/create_server.ipynb)
- [Delete Server](../modules-python/servers/delete_server.ipynb)
- [Create Network](../modules-python/network/create_network.ipynb)
- [Delete Network](../modules-python/network/delete_network.ipynb)
- [Create Subnet](../modules-python/network/create_subnet.ipynb)
- [Delete Subnet](../modules-python/network/delete_subnet.ipynb)

#### Import Library

```
import chi
from chi.server_api_examples import *
from chi.reservation_api_examples import *
from chi.networking_api_examples import *
```




## Tutorial: 

#### Configure the Environment

In [4]:
import json
import os
import chi
from datetime import datetime, timedelta
from dateutil import tz

from chi.server import *
from chi.lease import *
from chi.network import *

#Config with your project and site
chi.use_site('CHI@UC')
chi.set('project_name', 'CH-822154')

#Insert keypair name
key_name = 'my_chameleon_key'  # Change to your keypair

# Tip: Name resources with your username for easier identification
username = os.getenv("USER")
server_name = username+'Server'
network_name = username+'Net'
subnet_name = username+'Subnet'
router_name = username+'Router'
lease_name = username+'Lease'

#Server attributes
image_name='CC-CentOS7'
flavor_name='baremetal'
node_type="compute_haswell"
server_count=1
physical_network="physnet1"
BLAZAR_TIME_FORMAT = '%Y-%m-%d %H:%M'
DEFAULT_CIDR = '192.168.1.0/24'

AttributeError: module 'chi.network' has no attribute 'chi_wizard_create_network'

#### Create a Lease

In [12]:
# Set start/end date for lease
# Start one minute into future to avoid Blazar thinking lease is in past
# due to rounding to closest minute.
start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)

# Build list of reservations (in this case there is only one reservation)
reservation_list = []
add_node_reservation(reservation_list, count=server_count, node_type=node_type)
add_network_reservation(reservation_list, network_name=network_name, physical_network=physical_network)
add_fip_reservation(reservation_list, count=1)

# Create the lease
chi.blazar().lease.create(name=lease_name, 
                            start=start_date,
                            end=end_date,
                            reservations=reservation_list, events=[])

#Get the lease by name
lease = get_lease(lease_name)
    
#Print the lease info
print(json.dumps(lease, indent=2))

RuntimeError: Error: Found 2 leases with name pruthLease. Expected 1

#### Get the Reservations

In [ ]:
#Get the lease by name
lease = get_lease_by_name(lease_name)

compute_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'physical:host', lease['reservations']))[0]['id']
network_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'network', lease['reservations']))[0]['id']
floatingip_reservation_id = list(filter(lambda reservation: reservation['resource_type'] == 'virtual:floatingip', lease['reservations']))[0]['id']

print("compute_reservation_id: " + compute_reservation_id)
print("network_reservation_id: " + network_reservation_id)
print("floatingip_reservation_id: " + floatingip_reservation_id)

#### Configute the Network

In [ ]:
network = get_network(network_name)
network_id = network['id']

print('Network ID: ' + str(network_id))

In [ ]:
subnet = create_subnet(subnet_name, network_id, cidr=DEFAULT_CIDR, gateway_ip=None)
router = create_router(router_name, gw_network_name='public')
attach_router_to_subnet(router_name=router_name, subnet_name=subnet_name)

#### Start the Server

In [ ]:
#create the server
server = create_server(server_name, 
                       compute_reservation_id, 
                       key_name=key_name, 
                       network_id=network_id, 
                       network_name=network_name, 
                       nics=[], 
                       image_id=None, 
                       image_name=image_name, 
                       flavor_id=None, 
                       flavor_name=flavor_name, 
                       count=1)

#### Associate the Floating IP   (TODO: need to find floating_ip from the reservation that was just made)

In [ ]:
floating_ip = associate_floating_ip(server_name)

print('Floating IP: ' + str(floating_ip))

#### Delete Server

In [ ]:
delete_server(server['id'])

#### De-configure Network

In [ ]:
try:
    result = remove_subnet_from_router(router['id'], subnet['id'])
except Exception as e:
    print("remove_subnet_from_router error:" + str(e))
    pass

try:
    result = delete_router(router['id'])
except Exception as e:
    print("delete_router error: " + str(e))
    pass

try:
    result = delete_subnet(subnet['id'])
except Exception as e:
    print("delete_subnet error: " + str(e))
    pass

try:
    result = delete_network(network_id)
except Exception as e:
    print("delete_network error: " + str(e))
    pass

#### Release Lease

In [ ]:
delete_lease(lease_name)